In [3]:
!pip install tensorflow pillow
!pip install tensorflow opencv-python numpy
!pip install rembg
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install google-cloud-firestore
!pip install pyrebase4
!pip install firebase-admin

In [2]:
from rembg import remove
import requests
from PIL import Image, ImageDraw, ImageFont, ImageColor
from io import BytesIO
import cv2
import numpy as np
import tensorflow as tf
import os
from google.colab import drive
import random
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

In [4]:
os.makedirs('original', exist_ok=True)
os.makedirs('masked', exist_ok=True)

In [5]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
img_url = 'https://asset.kompas.com/crops/k3rnh0TPCkEd900x8n81MaYErXE=/0x0:1000x667/750x500/data/photo/2021/08/07/610e1825a3000.jpg'
img_name = img_url.split('/')[-1]
img_name

'610e1825a3000.jpg'

In [7]:
img = Image.open(BytesIO(requests.get(img_url).content))
img.save('original/' + img_name, format='JPEG')
output_path = 'masked/'+img_name
output_path

'masked/610e1825a3000.jpg'

In [8]:
with open(output_path, 'wb') as f :
  input = open('original/'+img_name, 'rb').read()
  subject = remove(input, alpha_matting=True, alpha_matting_foreground_threshold=50)
  f.write(subject)

100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 116GB/s]


In [9]:

# Function to list files in a specific Google Drive folder
def list_files(folder_id):
    drive_service = build('drive', 'v3')
    query = f"'{folder_id}' in parents"
    results = drive_service.files().list(q=query).execute()
    files = results.get('files', [])
    return files

In [10]:
# Function to download a file from Google Drive
def download_file(file_id):
    drive_service = build('drive', 'v3')
    request = drive_service.files().get_media(fileId=file_id)
    file_content = MediaIoBaseDownload(BytesIO())
    downloader = MediaIoBaseDownload(file_content, request)
    done = False
    while done is False:
        _, done = downloader.next_chunk()
    return file_content.getvalue()

In [12]:
back_img = 'https://storage.googleapis.com/capstone-project-406403/template-produk/produk-8.png'
back_img = Image.open(BytesIO(requests.get(back_img).content))

back_img = back_img.resize((1080, 1080))

back_img = back_img.convert('RGB')  # Convert RGBA to RGB

fore_img = Image.open(output_path)

fore_img = fore_img.resize((840, 540))

background_array = np.array(back_img)
background_array = cv2.cvtColor(background_array, cv2.COLOR_BGR2RGB)

# informasi Deteksi Warna

hex_color_img = '#3D2B1F'
hex_color_nama_makanan = '#CCCCFF'
hex_color_harga= '#801818'
hex_color_daerah= '#6082B6'
hex_color_promosi= '#FFC40C'
hex_color_instagram= '#645452'
hex_color_tiktok= '#003399'
hex_color_whatsapp= '#92000A'

# Convert hexadecimal to BGR
def hex_to_rgb_array(hex_color):
    return np.array([int(hex_color[i:i+2], 16) for i in (5, 3, 1)])

color_img = np.array(hex_to_rgb_array(hex_color_img))
color_nama_makanan = np.array(hex_to_rgb_array(hex_color_nama_makanan))
color_harga = np.array(hex_to_rgb_array(hex_color_harga))
color_daerah = np.array(hex_to_rgb_array(hex_color_daerah))
color_promosi = np.array(hex_to_rgb_array(hex_color_promosi))
color_instagram = np.array(hex_to_rgb_array(hex_color_instagram))
color_tiktok = np.array(hex_to_rgb_array(hex_color_tiktok))
color_whatsapp = np.array(hex_to_rgb_array(hex_color_whatsapp))

# set gambar produk
diff_img = np.sqrt(np.sum((background_array - color_img) ** 2, axis=2))
min_indices_img = np.unravel_index(np.argmin(diff_img), diff_img.shape)

x_offset_img = min_indices_img[1] - fore_img.width // 2
y_offset_img = min_indices_img[0] - fore_img.height // 2

back_img.paste(fore_img, (x_offset_img, y_offset_img), fore_img)

# Calculate the dominant color in the background image
def calculate_dominant_color(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Calculate the mean of pixel values
    mean_value = np.mean(gray)
    if mean_value > 200:  # If predominantly white
        return 'white'
    else:
        return 'black'

# Fungsi untuk menambahkan teks ke gambar
def add_redKits(image,dominant_color, text, x, y, font_size):
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("/content/drive/MyDrive/RedKits.otf", font_size)
    if dominant_color == 'white':
        draw.text((x, y), text, fill=(0, 0, 0), font=font)  # Set text to black
    else:
        draw.text((x, y), text, fill=(255, 255, 255), font=font)  # Default: white text
    return draw.textsize(text, font=font)

# Fungsi untuk menambahkan teks ke gambar
def add_poppins_white(image, text, x, y, font_size):
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("/content/drive/MyDrive/FontsFree-Net-Poppins-SemiBold.ttf", font_size)
    draw.text((x, y), text, fill=(255, 255, 255), font=font)
    return draw.textsize(text, font=font)

# Fungsi untuk menambahkan teks ke gambar
def add_poppins_black(image, text, x, y, font_size):
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("/content/drive/MyDrive/FontsFree-Net-Poppins-SemiBold.ttf", font_size)
    draw.text((x, y), text, fill=(0, 0, 0), font=font)
    return draw.textsize(text, font=font)

dominant_color = calculate_dominant_color(background_array)
# set tulisan nama produk
diff_nama_makanan = np.sqrt(np.sum((background_array - color_nama_makanan) ** 2, axis=2))
min_indices_nama_makanan = np.unravel_index(np.argmin(diff_nama_makanan), diff_nama_makanan.shape)

text_nama_makanan = "Kue Beras"
font_size_nama_makanan = 100

x_offset_nama_makanan = min_indices_nama_makanan[1] - fore_img.width//3
y_offset_nama_makanan = min_indices_nama_makanan[0] - font_size_nama_makanan//2

add_redKits(back_img, dominant_color, text_nama_makanan, x_offset_nama_makanan, y_offset_nama_makanan, font_size_nama_makanan)

# set tulisan harga produk
diff_harga= np.sqrt(np.sum((background_array - color_harga) ** 2, axis=2))
min_indices_harga = np.unravel_index(np.argmin(diff_harga), diff_harga.shape)

text_harga = "20.000"
font_size_harga = 40

x_offset_harga = min_indices_harga[1] - 5
y_offset_harga = min_indices_harga[0] - font_size_harga//2

add_poppins_white(back_img, text_harga, x_offset_harga, y_offset_harga, font_size_harga)

# set tulisan daerah produk
diff_daerah = np.sqrt(np.sum((background_array - color_daerah) ** 2, axis=2))
min_indices_daerah = np.unravel_index(np.argmin(diff_daerah), diff_daerah.shape)

text_daerah = "Medan"
font_size_daerah= 90

x_offset_daerah = min_indices_daerah[1] - 10
y_offset_daerah = min_indices_daerah[0] - font_size_daerah//3

add_redKits(back_img, dominant_color, text_daerah, x_offset_daerah, y_offset_daerah, font_size_daerah)

# set tulisan kata-kata promosi
diff_promosi = np.sqrt(np.sum((background_array - color_promosi) ** 2, axis=2))
min_indices_promosi = np.unravel_index(np.argmin(diff_promosi), diff_promosi.shape)

text_promosi = "Rencah Di Mulut Murah Harganya"
font_size_promosi= 60

x_offset_promosi = min_indices_promosi[1] - font_size_promosi//4
y_offset_promosi = min_indices_promosi[0] - font_size_promosi//2

if len(text_promosi) > 20:
    lines = []
    words = text_promosi.split()
    current_line = ''
    for word in words:
        if len(current_line + word) <= 20:
            current_line += word + ' '
        else:
            lines.append(current_line)
            current_line = word + ' '
    lines.append(current_line)

    y_offset = y_offset_promosi
    for line in lines:
        text_width, text_height = add_redKits(back_img, dominant_color, line.strip(), x_offset_promosi, y_offset, font_size_promosi)
        y_offset += text_height

else:
    add_redKits(back_img, dominant_color, text_promosi, x_offset_promosi, y_offset_promosi, font_size_promosi)

# set tulisan instagram
diff_instagram= np.sqrt(np.sum((background_array - color_instagram) ** 2, axis=2))
min_indices_instagram = np.unravel_index(np.argmin(diff_instagram), diff_instagram.shape)

text_instagram = "@Paskita"
font_size_instagram = 12

x_offset_instagram = min_indices_instagram[1]-font_size_instagram//4
y_offset_instagram = min_indices_instagram[0]-4

add_poppins_black(back_img, text_instagram, x_offset_instagram, y_offset_instagram, font_size_instagram)

# set tulisan tiktok
diff_tiktok= np.sqrt(np.sum((background_array - color_tiktok) ** 2, axis=2))
min_indices_tiktok = np.unravel_index(np.argmin(diff_tiktok), diff_tiktok.shape)

text_tiktok = "Paskita"
font_size_tiktok = 12

x_offset_tiktok = min_indices_tiktok[1]-font_size_tiktok//4
y_offset_tiktok = min_indices_tiktok[0]- 4

add_poppins_black(back_img, text_tiktok, x_offset_tiktok, y_offset_tiktok, font_size_tiktok)

# set tulisan whatsapp
diff_whatsapp= np.sqrt(np.sum((background_array - color_whatsapp) ** 2, axis=2))
min_indices_whatsapp = np.unravel_index(np.argmin(diff_whatsapp), diff_whatsapp.shape)

text_whatsapp = "082145863515"
font_size_whatsapp = 12

x_offset_whatsapp = min_indices_whatsapp[1]-font_size_whatsapp//4
y_offset_whatsapp = min_indices_whatsapp[0]-4

add_poppins_black(back_img, text_whatsapp, x_offset_whatsapp, y_offset_whatsapp, font_size_whatsapp)

back_img.save('masked/background.jpg', format='JPEG')

<ipython-input-12-663ba0693192>:67: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  return draw.textsize(text, font=font)
<ipython-input-12-663ba0693192>:74: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  return draw.textsize(text, font=font)
<ipython-input-12-663ba0693192>:67: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  return draw.textsize(text, font=font)
<ipython-input-12-663ba0693192>:67: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  return draw.textsize(text, font=font)
<ipython-input-12-663ba0693192>:67: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  return draw.textsize(text, f

# New Section

In [13]:
from google.colab import files

files.download( '/content/masked/background.jpg', )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>